### Load the packages

In [1]:
library(rstan)
library(bayesplot)
library(ggplot2)
library(doParallel)
library(devtools)
Sys.setenv(PATH = paste("C:/rtools40/mingw64/bin/", Sys.getenv("PATH"), sep=";"))
Sys.setenv(BINPREF = "C:/rtools40/mingw64/bin/")

載入需要的套件：StanHeaders

載入需要的套件：ggplot2

rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Do not specify '-march=native' in 'LOCAL_CPPFLAGS' or a Makevars file

This is bayesplot version 1.8.1

- Online documentation and vignettes at mc-stan.org/bayesplot

- bayesplot theme set to bayesplot::theme_default()

   * Does _not_ affect other ggplot2 plots

   * See ?bayesplot_theme_set for details on theme setting

載入需要的套件：foreach

載入需要的套件：iterators

載入需要的套件：parallel

載入需要的套件：usethis



In [2]:
rstan_options(auto_write=TRUE)
options(mc.cores=parallel::detectCores())
registerDoParallel(4)
getDoParWorkers()

[1] 4

### Define a function to prepare the parameters and data for further Bayesian analysis

In [3]:
getpara<-function(filename=NULL,names.mk=NULL){
          data.mk<-read.csv(filename)
          names.study<-unique(fixlen.str(data.mk$Study))
          hash.study<-1:length(names.study)
          names(hash.study)<-names.study
          hash.mk<-1:length(names.mk)
          names(hash.mk)<-names.mk
          para<-data.frame(study.names=fixlen.str(data.mk$Study),
          Study=array(hash.study[as.character(fixlen.str(data.mk$Study))]),
                      TP=round(data.mk$Case*data.mk$Sensitivity),
                      TN=round(data.mk$Control*data.mk$Specificity),
                      Dis=data.mk$Case,
                      NDis=data.mk$Control,
                      Test=array(hash.mk[as.character(data.mk$Marker)]))
    return(list(para=para,data=data.mk))
  }


### Define a function to get the upper and lower credible bound (97.5%)

In [4]:
get.ub.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.975)])
  }

get.lb.95<-function(x){
            y=sort(x)
            len=length(x)
            return(y[round(len*0.025)])
  }

### Define a function to get the results from Network Meta-analysis

In [5]:
get.nma<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	        data.nma<-data.ls$data.nma
	        func.ub<-NULL
        	func.lb<-NULL
	        if(conf==95){
		                    func.ub<-get.ub.95
		                    func.lb<-get.lb.95
	        } else if(conf==90){
		                           func.ub<-get.ub.90
		                           func.lb<-get.lb.90
	        }
	        data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	        data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
            data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	        mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	        rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	        or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                            ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                            lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                            mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                            ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                            lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	        dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                             ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                             lb=array(unlist(lapply(data.nma.dor,func.lb))))
            sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                                ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                                lb=array(unlist(lapply(data.nma.sindex,func.lb))))
            sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
            sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	        results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                          mu=mu.info,
	                          rr=rr.info,
	                          or=or.info,
	                          dor=dor.info,
                              sindex=sindex.info,
                              sindex_se=sindex_se.info,
                              sindex_sp=sindex_sp.info)
	   return(results.nma)
}

### Define a function to extract results from analysis

In [6]:
fixlen.str<-function(string.arr=NULL){
                                     	 max.len<-max(nchar(as.character(string.arr)))
	                                     rule<-paste('% -',as.character(max.len),'s',sep='')
	                                     string.arr.fix<-sprintf(rule,string.arr)
	                                     return(string.arr.fix)
}
fit.extract<-function(fit.data=NULL,filename.save=NULL){
	                                                       results.sum<-summary(fit.data)[[1]]
	                                                       results.simdata<-fit.data@sim$samples[[1]]
	                                                       save(results.sum,results.simdata,file=filename.save)
	                                                       return(results.simdata)
}

##########################################################
###  Network Meta-analysis (stan code)
##########################################################

In [7]:
stan.code.nma<-'
data{
     int N;  //number of comparison??? - 121
     int Nt; //number of test - 10
     int Ns; //number of study - 72
     int TP[N];
     int Dis[N];  //diseased
     int TN[N];
     int NDis[N]; //non-diseased
     int Study[N];
     int Test[N];
}
parameters{
           matrix[2, Nt] logitmu;
           vector[Ns] nu[2];
           matrix[Ns, Nt] delta[2];
           vector<lower=0>[Nt] tau[2]; //*
           vector<lower=0>[2] sigmab; 
           real<lower=-1, upper=1> rho;
}
transformed parameters{
                       matrix[Ns, 2] p_i[Nt];
                       matrix[2, Nt] MU;
                       matrix[2, Nt] RR;
                       matrix[2, Nt] OR;
                       vector[Nt] DOR;
                       vector[Nt] S;
                       vector[Nt] S_se;
                       vector[Nt] S_sp;
                       matrix[Nt, Nt] A;
                       matrix[Nt, Nt] B;
                       matrix[Nt, Nt] C;
                       matrix[Nt, Nt] A_se;
                       matrix[Nt, Nt] B_se;
                       matrix[Nt, Nt] C_se;
                       matrix[Nt, Nt] A_sp;
                       matrix[Nt, Nt] B_sp;
                       matrix[Nt, Nt] C_sp;
                       vector<lower=0>[Nt] tausq[2];
                       vector<lower=0>[2] sigmabsq;
                       matrix[Nt, Nt] sigmasq[2];
                       matrix[Nt, Nt] rhow[2];

    for (i in 1:Ns){
        for (j in 1:2){
            for (k in 1:Nt)
                p_i[k][i,j] = inv_logit(logitmu[j,k] +  nu[j][i] + delta[j][i,k]);
        }
    }
 
    for (j in 1:2){
        for (k in 1:Nt){
                         MU[j,k] = mean(col(p_i[k], j));
        }
        tausq[j] = (tau[j]).*(tau[j]);
    }

    for (j in 1:2){
        for (k in 1:Nt){
                         RR[j, k] = MU[j, k]/MU[j, 1]; 
                         OR[j, k] = (MU[j, k]*(1 - MU[j, 1]))/(MU[j, 1]*(1 - MU[j, k]));
         }
    }

    for (l in 1:Nt){
                     DOR[l] = (MU[1, l]*MU[2, l])/((1 - MU[1, l])*(1 - MU[2, l]));

        for(m in 1:Nt){
                        A[l, m] = if_else((MU[1, l] > MU[1, m]) && (MU[2, l] > MU[2, m]), 1, 0);
                        B[l, m] = if_else((MU[1, l] < MU[1, m]) && (MU[2, l] < MU[2, m]), 1, 0);
                        C[l, m] = if_else((MU[1, l] == MU[1, m]) && (MU[2, l] == MU[2, m]), 1, 0);

                        A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
                        B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
                        C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);

                        A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
                        B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
                        C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
        }

        S[l] = (2*sum(row(A, l)) + sum(row(C, l)))/(2*sum(row(B, l)) + sum(row(C, l)));
        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
    }
    
    sigmabsq = (sigmab).*(sigmab);

    for (j in 1:2){
        for (k in 1:Nt){
            for (l in 1:Nt){
                             sigmasq[j][k,l] = (sigmabsq[j] + tausq[j][k])*((sigmabsq[j] + tausq[j][l]));
                             rhow[j][k,l] = sigmabsq[j]/sqrt(sigmasq[j][k,l]);
            }
        }
    }

}
model{
	   //Priors
       for (j in 1:2){
                       logitmu[j] ~ normal(0, 5);
		               tau[j] ~ cauchy(0, 2.5);
       }

         sigmab ~ cauchy(0, 2.5);
	     rho ~ uniform(-1, 1);
         nu[2] ~ normal(0, sigmab[2]);
      
       for (i in 1:Ns){
                       nu[1][i] ~ normal((sigmab[1]/sigmab[2])*rho*nu[2][i], sqrt(sigmabsq[1]*(1 - (rho*rho))));
          for (j in 1:2){
              for (k in 1:Nt)
                              delta[j][i,k] ~ normal(0, tau[j][k]);
        }
    }

    for (n in 1:N){
                    TP[n] ~ binomial(Dis[n], p_i[Test[n]][Study[n], 1]);
                    TN[n] ~ binomial(NDis[n], p_i[Test[n]][Study[n], 2]);
    }

}
generated quantities{
    
    vector[2*N] loglik;

    for (n in 1:N)
                   loglik[n] = binomial_lpmf(TN[n]| NDis[n], p_i[Test[n]][Study[n], 1]);

    for (n in (N+1):(2*N))
                   loglik[n] = binomial_lpmf(TN[n-N]| NDis[n-N], p_i[Test[n-N]][Study[n-N], 2]);

}
'

### Lists of Biomarkers in network meta-analysis 

In [8]:
mk.names=c('qSOFA',
           'PCT',
           'presepsin',
           'CRP',
           'CD64',
           'IL-6',
           'sTREM-1',
           'LBP',
           'SIRS',
           'SOFA')

### Data set to used

In [9]:
filename<-'Sepsis_nma_data.csv'

### Parameters lists

In [10]:
para.ls<-getpara(filename=filename,names.mk=mk.names)
para<-para.ls$para
data<-para.ls$data

### Prepare data for Network Meta-analysis

In [11]:
para.as.ls.nma<-list(N=nrow(para),
                     Nt=length(unique(para$Test)),
                     Ns=length(unique(para$Study)),
                     TP=para$TP,
                     Dis=para$Dis,
                     TN=para$TN,
                     NDis=para$NDis,
                     Study=as.numeric(para$Study),
                     Test=para$Test)

### Compiling model

In [12]:
stan.model.nma<-stan(model_code=stan.code.nma,
                     data=para.as.ls.nma, 
                     iter=1, 
                     warmup=0, 
                     chains=2)

Warning message:
"There were 2 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"


### Running model

In [13]:
start_time <- Sys.time()
fit.nma<-stan(fit=stan.model.nma,
              data=para.as.ls.nma,
              iter=20000,
              chains=4,
              cores=4,
              control=list(adapt_delta=0.99,
                           stepsize=0.01,
                           max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

Warning message:
"There were 2 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat"
Warning message:
"Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess"
Warning message:
"Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more 

Time difference of 21.05406 hours

### Save the results as .RData and .csv

In [14]:
filename.simdata.save<-'fit_nma_simdata_qSOFAasreference.RData'
filename.results.save<-'out_nma_sepsis3_qSOFAasreference.csv'
results.nma<-fit.extract(fit.data=fit.nma,filename.save=filename.simdata.save)
results.nma.95<-get.nma(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.nma),mk.names=mk.names,
                            conf=95)
write.csv(results.nma.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-analysis

In [15]:
rownames(results.nma.95$mu)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$mu

,mean_sp,ub_sp,lb_sp,mean_se,ub_se,lb_se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
qSOFA,0.9212462,0.9854525,0.6234143,0.3462950,0.5890535,0.1754487
PCT,0.7477665,0.7719957,0.7205840,0.7354603,0.7612342,0.7070550
presepsin,0.7319718,0.7921475,0.6580232,0.7985793,0.8486892,0.7358911
CRP,0.7016000,0.7439267,0.6556856,0.6789026,0.7219475,0.6316500
CD64,0.7866743,0.9091934,0.5823358,0.7801413,0.9239502,0.5047990
IL-6,0.7468872,0.8425525,0.6313472,0.7223712,0.7974009,0.6356649
sTREM-1,0.7895254,0.9329969,0.4474239,0.5089890,0.7851218,0.2318868
LBP,0.5947912,0.8235476,0.2964033,0.6194268,0.8511927,0.2949110
SIRS,0.5288711,0.6715261,0.3799266,0.7652010,0.8698112,0.5906220


### Print the results of pooled DOR from Network Meta-analysis

In [16]:
rownames(results.nma.95$dor)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$dor

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,13.526483,45.181787,0.8033880
PCT,8.300979,9.960500,6.8406802
presepsin,11.327114,17.527737,6.7170227
CRP,5.045064,6.621215,3.7443194
CD64,20.561410,65.586930,3.0141806
IL-6,8.383418,15.533617,3.9951144
sTREM-1,6.558606,24.048878,0.5683738
LBP,3.624429,12.861587,0.4134923
SIRS,4.214887,8.940849,1.4313695


### Print the results of pooled Superiority index from Network Meta-analysis

In [17]:
rownames(results.nma.95$sindex)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,1.3767658,5,0.06666667
PCT,3.0181405,9,0.60000000
presepsin,5.8115763,13,1.00000000
CRP,0.4198025,1,0.09090909
CD64,9.3230095,17,0.20000000
IL-6,3.1167224,11,0.14285714
sTREM-1,1.6822008,11,0.05882353
LBP,0.5794632,5,0.05882353
SIRS,0.6611422,3,0.07692308


### Print the results of pooled Superiority index for Sensitivity from Network Meta-analysis

In [18]:
rownames(results.nma.95$sindex_se)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_se

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,0.1001510,0.3333333,0.05263158
PCT,2.0739580,5.6666667,0.81818182
presepsin,8.9231022,19.0000000,1.85714286
CRP,0.8571430,1.8571429,0.33333333
CD64,9.6899868,19.0000000,0.33333333
IL-6,2.0272871,5.6666667,0.53846154
sTREM-1,0.6012746,3.0000000,0.05263158
LBP,1.6826068,19.0000000,0.05263158
SIRS,6.3423378,19.0000000,0.53846154


### Print the results of pooled Superiority index for Specificity from Network Meta-analysis

In [19]:
rownames(results.nma.95$sindex_sp)=c('qSOFA','PCT','presepsin','CRP','CD64','IL-6','sTREM-1','LBP','SIRS','SOFA')
results.nma.95$sindex_sp

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,16.3646223,19.0000000,0.33333333
PCT,1.2382790,3.0000000,0.53846154
presepsin,1.0221550,3.0000000,0.17647059
CRP,0.4962017,1.2222222,0.17647059
CD64,3.1887404,19.0000000,0.17647059
IL-6,1.4958993,5.6666667,0.17647059
sTREM-1,4.2778313,19.0000000,0.05263158
LBP,0.4658956,3.0000000,0.05263158
SIRS,0.1140790,0.3333333,0.05263158


##################################################################################
###                          Univariable Network meta-regression                         ###
##################################################################################

### Define a function to get the results from Univariable Network Meta-regression

In [20]:
get.nma.1mk<-function(data.ls=NULL,mk.names=NULL,n.study=1,conf=95){
	           data.nma<-data.ls$data.nma
	           func.ub<-NULL
	           func.lb<-NULL
	           if(conf==95){
		                      func.ub<-get.ub.95
		                      func.lb<-get.lb.95
	           } else if(conf==90){
		                             func.ub<-get.ub.90
		                             func.lb<-get.lb.90
	           }
	           data.nma.muse<-data.nma[paste('MU[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.musp<-data.nma[paste('MU[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrse<-data.nma[paste('RR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.rrsp<-data.nma[paste('RR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orse<-data.nma[paste('OR[',1,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.orsp<-data.nma[paste('OR[',2,',',1:length(data.ls$mk),']',sep='')]
	           data.nma.dor<-data.nma[paste('DOR[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex<-data.nma[paste('S[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_se<-data.nma[paste('S_se[',1:length(data.ls$mk),']',sep='')]
	           data.nma.sindex_sp<-data.nma[paste('S_sp[',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1se<-data.nma[paste('beta1[1,',1:length(data.ls$mk),']',sep='')]
	           data.nma.beta1sp<-data.nma[paste('beta1[2,',1:length(data.ls$mk),']',sep='')]
	           mu.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.musp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.musp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.musp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.muse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.muse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.muse,func.lb))))
	           rr.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.rrsp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.rrsp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.rrsp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.rrse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.rrse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.rrse,func.lb))))
	           or.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.orsp,mean))),
	                               ub_sp=array(unlist(lapply(data.nma.orsp,func.ub))),
	                               lb_sp=array(unlist(lapply(data.nma.orsp,func.lb))),
	                               mean_se=array(unlist(lapply(data.nma.orse,mean))),
	                               ub_se=array(unlist(lapply(data.nma.orse,func.ub))),
	                               lb_se=array(unlist(lapply(data.nma.orse,func.lb))))
	           dor.info<-data.frame(mean=array(unlist(lapply(data.nma.dor,mean))),
	                                ub=array(unlist(lapply(data.nma.dor,func.ub))),
	                                lb=array(unlist(lapply(data.nma.dor,func.lb))))
	           sindex.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex,mean))),
	                                   ub=array(unlist(lapply(data.nma.sindex,func.ub))),
	                                   lb=array(unlist(lapply(data.nma.sindex,func.lb))))
	           sindex_se.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_se,mean))),
	                                      ub=array(unlist(lapply(data.nma.sindex_se,func.ub))),
	                                      lb=array(unlist(lapply(data.nma.sindex_se,func.lb))))
	           sindex_sp.info<-data.frame(mean=array(unlist(lapply(data.nma.sindex_sp,mean))),
	                                      ub=array(unlist(lapply(data.nma.sindex_sp,func.ub))),
	                                      lb=array(unlist(lapply(data.nma.sindex_sp,func.lb))))
	           beta1.info<-data.frame(mean_sp=array(unlist(lapply(data.nma.beta1sp,mean))),
	                                  ub_sp=array(unlist(lapply(data.nma.beta1sp,func.ub))),
	                                  lb_sp=array(unlist(lapply(data.nma.beta1sp,func.lb))),
	                                  mean_se=array(unlist(lapply(data.nma.beta1se,mean))),
	                                  ub_se=array(unlist(lapply(data.nma.beta1se,func.ub))),
	                                  lb_se=array(unlist(lapply(data.nma.beta1se,func.lb))))
	           results.nma<-list(marker=mk.names[mk.names %in% data.ls$mk],
	                             mu=mu.info,
	                             rr=rr.info,
	                             or=or.info,
	                             dor=dor.info,
	                             sindex=sindex.info,
	                             sindex_se=sindex_se.info,
	                             sindex_sp=sindex_sp.info,
	                             beta1=beta1.info)
	return(results.nma)
}

### Univariable Network Meta-regression (STAN code)

In [24]:
stan.code.1mk<-'
                data{
                      int N;         //number of comparison (n=121)
                      int Nt;        //number of test (n=7)
                      int Ns;        //number of study (n=107)
                      int TP[N];
                      int Dis[N];    //diseased
                      int TN[N];
                      int NDis[N];   //non-diseased
                      int Study[N];
                      int Test[N];
                      int Covar1[N]; //one additional variable
                    }

                parameters{
                            matrix[2,Nt] logitmu;
                            vector[Ns] nu[2];
                            matrix[Ns,Nt] delta[2];
                            vector<lower=0>[Nt] tau[2];
                            vector<lower=0>[2] sigmab; 
                            real<lower=-1,upper=1> rho;
                            matrix[2,Nt] beta1;
                        }

                transformed parameters{
                                        matrix[Ns,2] p_i[Nt];
                                        matrix[2,Nt] MU;
                                        matrix[2,Nt] RR;
                                        matrix[2,Nt] OR;
                                        vector[Nt] DOR;
                                        vector[Nt] S;
                                        vector[Nt] S_se;
                                        vector[Nt] S_sp;
                                        matrix[Nt,Nt] A;
                                        matrix[Nt,Nt] B;
                                        matrix[Nt,Nt] C;
                                        matrix[Nt, Nt] A_se;
                                        matrix[Nt, Nt] B_se;
                                        matrix[Nt, Nt] C_se;
                                        matrix[Nt, Nt] A_sp;
                                        matrix[Nt, Nt] B_sp;
                                        matrix[Nt, Nt] C_sp;

                                        vector<lower=0>[Nt] tausq[2];
                                        vector<lower=0>[2] sigmabsq;

                                        matrix[Nt,Nt] sigmasq[2];
                                        matrix[Nt,Nt] rhow[2];

                for (i in 1:Ns){
                     for (j in 1:2){
                          for (k in 1:Nt){
                                           p_i[k][i,j]=inv_logit(logitmu[j,k]+nu[j][i]+delta[j][i,k]+beta1[j,k]*Covar1[i]);
                           }
                      }
                 }

                for (j in 1:2){
                     for (k in 1:Nt){
                                      MU[j,k]=mean(col(p_i[k],j));
                     }
                      tausq[j]=(tau[j]).*(tau[j]);
                }
                  
                for (j in 1:2){
                     for (k in 1:Nt){
                                      RR[j,k]=MU[j,k]/MU[j,1]; 
                                      OR[j,k]=(MU[j,k]*(1-MU[j,1]))/(MU[j,1]*(1-MU[j,k]));
                       }
                 }

                for (l in 1:Nt){
                                 DOR[l]=(MU[1,l]*MU[2,l])/((1-MU[1,l])*(1-MU[2,l]));
                     for(m in 1:Nt){
                                     A[l,m]=if_else((MU[1,l]>MU[1,m]) && (MU[2,l]>MU[2,m]),1,0);
                                     B[l,m]=if_else((MU[1,l]<MU[1,m]) && (MU[2,l]<MU[2,m]),1,0);
                                     C[l,m]=if_else((MU[1,l]==MU[1,m]) && (MU[2,l]==MU[2,m]),1,0);
                                     
                                     A_se[l, m] = if_else((MU[1, l] > MU[1, m]), 1, 0);
                                     B_se[l, m] = if_else((MU[1, l] < MU[1, m]), 1, 0);
                                     C_se[l, m] = if_else((MU[1, l] == MU[1, m]), 1, 0);
 
                                     A_sp[l, m] = if_else((MU[2, l] > MU[2, m]), 1, 0);
                                     B_sp[l, m] = if_else((MU[2, l] < MU[2, m]), 1, 0);
                                     C_sp[l, m] = if_else((MU[2, l] == MU[2, m]), 1, 0);
                      }
                        S[l]=(2*sum(row(A,l))+sum(row(C,l)))/(2*sum(row(B,l))+sum(row(C,l)));
                        S_se[l] = (2*sum(row(A_se, l)) + sum(row(C_se, l)))/(2*sum(row(B_se, l)) + sum(row(C_se, l)));
                        S_sp[l] = (2*sum(row(A_sp, l)) + sum(row(C_sp, l)))/(2*sum(row(B_sp, l)) + sum(row(C_sp, l)));
                }
        
                 sigmabsq=(sigmab).*(sigmab);

                for (j in 1:2){
                     for (k in 1:Nt){
                          for (l in 1:Nt){
                                           sigmasq[j][k,l]=(sigmabsq[j]+tausq[j][k])*((sigmabsq[j]+tausq[j][l]));
                                           rhow[j][k,l]=sigmabsq[j]/sqrt(sigmasq[j][k,l]);
                            }
                       }
                 }
            }

                model{
                      //Priors
                      for (j in 1:2){
                                      logitmu[j]~normal(0,5);
                                      tau[j]~cauchy(0,2.5);
                                      
                           for (k in 1:Nt){
                                            beta1[j,k]~normal(0,5);
                           }            
                      }
                       sigmab~cauchy(0, 2.5);
                       rho~uniform(-1, 1);
                       nu[2]~normal(0,sigmab[2]);

                      for (i in 1:Ns){
                                       nu[1][i]~normal((sigmab[1]/sigmab[2])*rho*nu[2][i],sqrt(sigmabsq[1]*(1-(rho*rho))));
                           for (j in 1:2){
                               for (k in 1:Nt){
                                                delta[j][i,k]~normal(0,tau[j][k]);
                              }
                         }
                      }

                     for (n in 1:N){
                                     TP[n]~binomial(Dis[n],p_i[Test[n]][Study[n],1]);
                                     TN[n]~binomial(NDis[n],p_i[Test[n]][Study[n],2]);
                      }

                    }

                generated quantities{
                                      vector[2*N] loglik;

                                      for (n in 1:N){
                                                      loglik[n]=binomial_lpmf(TN[n]|NDis[n],p_i[Test[n]][Study[n],1]);
                                       }
                                      for (n in (N+1):(2*N)){
                                                              loglik[n]=binomial_lpmf(TN[n-N]|NDis[n-N],p_i[Test[n-N]][Study[n-N],2]);
                                       }
                }
'

### Prepare data for Network Meta-regression

In [25]:
para.as.ls.1mk<-list(N=nrow(para),
                     Nt=length(unique(para$Test)),
                     Ns=length(unique(para$Study)),
                     TP=para$TP,
                     Dis=para$Dis,
                     TN=para$TN,
                     NDis=para$NDis,
                     Study=as.numeric(para$Study),
                     Test=para$Test,
                     Covar1=data$is.prevalence05)

### Compiling model

In [26]:
stan.model.1mk<-stan(model_code=stan.code.1mk, 
                     data=para.as.ls.1mk, 
                     iter=1, 
                     warmup=0, 
                     chains=1)


SAMPLING FOR MODEL 'ba28c30f4d7b4e109b9003b7ef4d575b' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: WARNING: No variance estimation is
Chain 1:          performed for num_warmup < 20
Chain 1: 
Chain 1: Iteration: 1 / 1 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0 seconds (Warm-up)
Chain 1:                0.002 seconds (Sampling)
Chain 1:                0.002 seconds (Total)
Chain 1: 


Warning message:
"There were 1 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"


### Running model

In [27]:
start_time <- Sys.time()
fit.1mk<-stan(fit=stan.model.1mk,
              data=para.as.ls.1mk,
              iter=20000,
              chains=4,
              cores=4,
              control=list(adapt_delta=0.99,
                           stepsize=0.01,
                           max_treedepth=17))
end_time <- Sys.time()
end_time - start_time

Warning message:
"There were 24 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 14 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 17. See
http://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded"
Warning message:
"There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
http://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is NA, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat"
Warning message:
"Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-s

Time difference of 1.026367 days

### Save the results as .RData and .csv

In [28]:
filename.simdata.save<-'fit_1mk_simdata_sepsis_isprevalence05.RData'
filename.results.save<-'out_1mk_sepsis_isprevalence05.csv'
results.1mk<-fit.extract(fit.data=fit.1mk,filename.save=filename.simdata.save)
results.1mk.95<-get.nma.1mk(data.ls=list(mk=mk.names[unique(para$Test)],
                                         data.nma=results.1mk),mk.names=mk.names,
                            conf=95)
write.csv(results.1mk.95,filename.results.save)

### Print the results of pooled sensitivity and specificity from Network Meta-regression

In [30]:
rownames(results.1mk.95$mu)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.1mk.95$mu

,mean_sp,ub_sp,lb_sp,mean_se,ub_se,lb_se
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
qSOFA,0.9215787,0.9856610,0.6169751,0.3706940,0.6623222,0.1631287
PCT,0.7475742,0.7728840,0.7188689,0.7328987,0.7597550,0.7029525
presepsin,0.7416470,0.7986596,0.6693620,0.7889227,0.8426478,0.7212468
CRP,0.6946711,0.7382630,0.6476014,0.6858907,0.7288520,0.6388682
CD64,0.7680605,0.9135454,0.5028803,0.7407672,0.9208603,0.4272572
IL-6,0.7440625,0.8201268,0.6507902,0.7199064,0.7931857,0.6342840
sTREM-1,0.6545852,0.9454023,0.2890427,0.5096032,0.8404560,0.1745123
LBP,0.5455414,0.8748910,0.1971126,0.5703920,0.8950256,0.2073530
SIRS,0.5314884,0.6730114,0.3739959,0.7640041,0.8688228,0.5822636


### Print the results of pooled DOR from Network Meta-regression

In [31]:
rownames(results.1mk.95$dor)=c('qSOFA', 'PCT', 'presepsin', 'CRP', 'CD64', 'IL-6', 'sTREM-1', 'LBP', 'SIRS', 'SOFA')
results.1mk.95$dor

,mean,ub,lb
,<dbl>,<dbl>,<dbl>
qSOFA,16.523532,64.010059,0.7677936
PCT,8.187895,9.856417,6.6985251
presepsin,11.225535,17.227293,6.7136439
CRP,5.042142,6.637642,3.7509636
CD64,16.962901,61.964684,1.8031082
IL-6,7.955965,13.364169,4.2801083
sTREM-1,6.634534,39.817375,0.1670906
LBP,4.622319,26.533325,0.1297156
SIRS,4.274367,9.188069,1.3487222
